Refer https://www.learnpytorch.io for further concepts on PyTorch

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision

from torchvision import transforms

from PIL import Image, ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES=True

In [2]:
print(torch.cuda.is_available())

True


In [3]:
!python3 download.py

Creating directory ./train/cat
Creating directory ./train/fish
Creating directory ./test/cat
Creating directory ./test/fish
Creating directory ./val/cat
Creating directory ./val/fish
Error downloading http://farm1.static.flickr.com/1/1004525_cba96ba3c3.jpg
Error downloading http://farm1.static.flickr.com/33/65048097_e5264bf855.jpg
Error downloading http://farm3.static.flickr.com/2354/2102976081_61c8614be8.jpg
Error downloading http://farm2.static.flickr.com/1390/709949156_5e4ac3f499.jpg
Error downloading http://farm1.static.flickr.com/81/243007380_feda8c8a4c.jpg
Error downloading http://farm2.static.flickr.com/1221/1011749126_44a195db4c.jpg
Error downloading http://farm1.static.flickr.com/173/416994740_6ada308baa.jpg
Error downloading http://farm1.static.flickr.com/3/4193130_a058cdb81f.jpg
Error downloading http://farm1.static.flickr.com/1/1004528_a111209743.jpg
Error downloading http://farm1.static.flickr.com/60/174732335_7275892317.jpg
Error downloading http://farm1.static.flickr.com

Error downloading http://farm1.static.flickr.com/27/57455726_8ccf14753f.jpg
Error downloading http://farm1.static.flickr.com/185/426029368_3a9612f006.jpg
Error downloading http://farm2.static.flickr.com/1133/1141687717_890fe14d8e.jpg
Error downloading http://farm1.static.flickr.com/62/204035942_de0d323af5.jpg
Error downloading http://farm1.static.flickr.com/221/513355862_0542a84fa0.jpg
Error downloading http://farm2.static.flickr.com/1172/1323667952_a7a74975c4.jpg
Error downloading http://farm1.static.flickr.com/42/84450056_bb5974a64f.jpg
Error downloading http://farm4.static.flickr.com/3001/2776498771_20527f258b.jpg
Error downloading http://farm4.static.flickr.com/3159/2824589595_07ee2443a3.jpg
Error downloading http://farm4.static.flickr.com/3206/2970698030_1021311f52.jpg
Error downloading http://farm2.static.flickr.com/1125/1348034256_cc50f5b446.jpg
Error downloading http://farm3.static.flickr.com/2416/1833444691_dc3d1017db.jpg
Error downloading http://farm3.static.flickr.com/2077/1

Error downloading http://farm2.static.flickr.com/1251/1084292263_542314f2ed.jpg
Error downloading http://farm4.static.flickr.com/3088/2294965469_d200313929.jpg
Error downloading http://farm4.static.flickr.com/3155/2897344310_0cb98bf481.jpg
Error downloading http://farm4.static.flickr.com/3219/3006747829_8658331ecd.jpg
Error downloading http://farm1.static.flickr.com/254/521168219_ad61ac2925.jpg
Error downloading http://farm1.static.flickr.com/32/50021271_c261cd4746.jpg
Error downloading http://farm3.static.flickr.com/2341/2021603270_2e53566a23.jpg
Error downloading http://farm4.static.flickr.com/3073/2791982836_fefc141d11.jpg
Error downloading http://farm2.static.flickr.com/1115/1388069914_0ecc61170d.jpg
Error downloading http://farm2.static.flickr.com/1302/1242460238_57b66f8072.jpg
Error downloading http://farm2.static.flickr.com/1273/1183434487_0de54007df.jpg
Error downloading http://farm1.static.flickr.com/97/273635025_5ca2b872b9.jpg
Error downloading http://farm2.static.flickr.com/

Error downloading http://www.alaskariveradventures.com/images/king%20salmon%20by%20stephanie%20holland.jpg
Error downloading http://www.bearsdenlodge.com/images/salmonjeff.jpg
Error downloading http://www.grayling-on-a-fly.com/images/man-salmon.jpg
Error downloading http://www.creeksideflyfishing.com/Salmon2003.jpg
Error downloading http://www.jimsfishing.com/images-photo_gallery/2004-08-23-02.jpg
Error downloading http://www.bristolbayadventures.com/images/silver_salmon_fishing_2.jpg
Error downloading http://alaskanadventures.biz/images/fishboat251.jpg
Error downloading http://www.gwinslodge.com/KENPHIL-SILVER.jpg
Error downloading http://i20.photobucket.com/albums/b218/grandpawrichard/Alaska%20Fishing%202008/AhappygroupwithsomeniceCohoSalmonsm.jpg
Error downloading http://www.guidebc.com/images/img0085_300.jpg
Error downloading http://www.zoo.utoronto.ca/manuesteve/cohodigging1.jpg
Error downloading http://www.flycastingguide.com/images/3478912/images/Chris%20with%20Coho%20Salmon.jpg

In [6]:
def check_image(path):
    try:
        im = Image.open(path)
        return True
    except:
        return False

### Image Transforms (Resize, Tensor Convertion, ImageNet mean, std)



In [7]:
img_transforms = transforms.Compose([
    transforms.Resize((64,64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #Avoid exploding gradient problem
])

In [8]:
train_data_path = "./train/"
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=img_transforms, is_valid_file=check_image)

In [11]:
val_data_path = "./val/"
val_data = torchvision.datasets.ImageFolder(root=val_data_path,transform=img_transforms, is_valid_file=check_image)

In [15]:
test_data_path = "./test/"
test_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=img_transforms, is_valid_file=check_image)

In [16]:
batch_size = 64

In [17]:
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

## Neural Network

In [19]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet,self).__init__()
        self.fc1 = nn.Linear(12288, 84)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)
        
    def forward(self):
        x = x.view(-1, 12288) # Convert to 1D Vector
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x
    
simplenet = SimpleNet()